In [3]:
!pip install transformers tqdm scikit-learn

import gzip
import random
import pandas as pd
import numpy as np
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import DistilBertTokenizer, DistilBertModel, AdamW
from tqdm import tqdm
import warnings



In [4]:
import pandas as pd

def load_checkins_txt(file_path):
    """
    Load Gowalla check-in data from a plain text file.
    """
    # Column names based on Gowalla data description
    columns = ["user_id", "check_in_time", "latitude", "longitude", "location_id"]
    data = pd.read_csv(file_path, sep="\t", names=columns)
    # Convert timestamp to datetime for easier manipulation
    data["check_in_time"] = pd.to_datetime(data["check_in_time"])
    return data

checkins_file = "/kaggle/input/raw-gowalla-checkins/Gowalla_totalCheckins.txt"  
gowalla_data = load_checkins_txt(checkins_file)
print(gowalla_data.head())


   user_id             check_in_time   latitude  longitude  location_id
0        0 2010-10-19 23:55:27+00:00  30.235909 -97.795140        22847
1        0 2010-10-18 22:17:43+00:00  30.269103 -97.749395       420315
2        0 2010-10-17 23:42:03+00:00  30.255731 -97.763386       316637
3        0 2010-10-17 19:26:05+00:00  30.263418 -97.757597        16516
4        0 2010-10-16 18:50:42+00:00  30.274292 -97.740523      5535878


In [5]:
def add_spatio_temporal_features(df, lat_multiplier=10, lon_multiplier=10):
    """
    Given a DataFrame with columns:
      - check_in_time (datetime)
      - latitude, longitude (floats)
    This function adds columns:
      - day_of_week
      - hour_of_day
      - lat_bucket
      - lon_bucket
    """
    df["day_of_week"] = df["check_in_time"].dt.dayofweek
    df["hour_of_day"] = df["check_in_time"].dt.hour
    
    # Example: Multiply lat/lon by 10, then floor to convert to integer buckets
    df["lat_bucket"] = (df["latitude"] * lat_multiplier).astype(int)
    df["lon_bucket"] = (df["longitude"] * lon_multiplier).astype(int)
    
    return df

gowalla_data = add_spatio_temporal_features(gowalla_data)
print(gowalla_data.head())


   user_id             check_in_time   latitude  longitude  location_id  \
0        0 2010-10-19 23:55:27+00:00  30.235909 -97.795140        22847   
1        0 2010-10-18 22:17:43+00:00  30.269103 -97.749395       420315   
2        0 2010-10-17 23:42:03+00:00  30.255731 -97.763386       316637   
3        0 2010-10-17 19:26:05+00:00  30.263418 -97.757597        16516   
4        0 2010-10-16 18:50:42+00:00  30.274292 -97.740523      5535878   

   day_of_week  hour_of_day  lat_bucket  lon_bucket  
0            1           23         302        -977  
1            0           22         302        -977  
2            6           23         302        -977  
3            6           19         302        -977  
4            5           18         302        -977  


In [6]:
def construct_trajectories(data, time_window="1D"):
    """
    Group check-ins into user trajectories based on a time window.
    time_window is a Pandas time frequency string (e.g., '1D' for daily).
    """
    trajectories = []

    grouped = data.groupby("user_id")
    for user_id, group in grouped:
        group = group.sort_values("check_in_time")
        # Identify new trajectory if gap > time_window
        group["trajectory_id"] = (group["check_in_time"].diff() > pd.Timedelta(time_window)).cumsum()

        for traj_id, traj_group in group.groupby("trajectory_id"):
            trajectory = {
                "user_id": user_id,
                "trajectory_id": traj_id,
                "timestamps": traj_group["check_in_time"].tolist(),
                "locations": traj_group["location_id"].tolist(),
                "day_of_week": traj_group["day_of_week"].tolist(),
                "hour_of_day": traj_group["hour_of_day"].tolist(),
                "lat_bucket": traj_group["lat_bucket"].tolist(),
                "lon_bucket": traj_group["lon_bucket"].tolist()
            }
            trajectories.append(trajectory)

    return pd.DataFrame(trajectories)

trajectories_df = construct_trajectories(gowalla_data, time_window="1D")
trajectories_df.head()


,user_id,trajectory_id,timestamps,locations,day_of_week,hour_of_day,lat_bucket,lon_bucket
0,0,0,"[2010-05-22 02:49:04+00:00, 2010-05-22 17:50:5...","[608105, 8977, 18574, 17269, 1161876, 1163401,...","[5, 5, 5, 6, 6, 6, 6]","[2, 17, 19, 16, 17, 22, 23]","[302, 392, 389, 390, 390, 390, 390]","[-977, -947, -946, -945, -945, -945, -945]"
1,0,1,[2010-05-26 14:16:56+00:00],[21714],[2],[14],[302],[-977]
2,0,2,[2010-05-27 22:39:52+00:00],[420315],[3],[22],[302],[-977]
3,0,3,[2010-05-30 06:17:57+00:00],[9073],[6],[6],[377],[-1223]
4,0,4,"[2010-05-31 20:10:37+00:00, 2010-06-01 14:38:2...","[18417, 480992, 15326, 420315]","[0, 1, 2, 2]","[20, 14, 0, 15]","[302, 302, 302, 302]","[-977, -977, -977, -977]"


In [7]:
def encode_pois(trajectories):
    """
    Encode POIs into unique numeric IDs.
    """
    all_locations = set(loc for traj in trajectories["locations"] for loc in traj)
    location_mapping = {loc: idx for idx, loc in enumerate(all_locations)}

    trajectories["encoded_locations"] = trajectories["locations"].apply(
        lambda locs: [location_mapping[loc] for loc in locs]
    )
    return trajectories, location_mapping

trajectories_df, poi_mapping = encode_pois(trajectories_df)
print(trajectories_df.head())

# trajectories_df.to_csv("gowalla_trajectories_spatiotemp.csv", index=False)



   user_id  trajectory_id                                         timestamps  \
0        0              0  [2010-05-22 02:49:04+00:00, 2010-05-22 17:50:5...   
1        0              1                        [2010-05-26 14:16:56+00:00]   
2        0              2                        [2010-05-27 22:39:52+00:00]   
3        0              3                        [2010-05-30 06:17:57+00:00]   
4        0              4  [2010-05-31 20:10:37+00:00, 2010-06-01 14:38:2...   

                                           locations            day_of_week  \
0  [608105, 8977, 18574, 17269, 1161876, 1163401,...  [5, 5, 5, 6, 6, 6, 6]   
1                                            [21714]                    [2]   
2                                           [420315]                    [3]   
3                                             [9073]                    [6]   
4                     [18417, 480992, 15326, 420315]           [0, 1, 2, 2]   

                   hour_of_day              

In [4]:
# loaded saved preprocessed trajectories
trajectories_df = pd.read_csv("/kaggle/input/spatio/gowalla_trajectories_spatiotemp.csv")

In [6]:
print(trajectories_df.head())

   user_id  trajectory_id                                         timestamps  \
0        0              0  [Timestamp('2010-05-22 02:49:04+0000', tz='UTC...   
1        0              1  [Timestamp('2010-05-26 14:16:56+0000', tz='UTC')]   
2        0              2  [Timestamp('2010-05-27 22:39:52+0000', tz='UTC')]   
3        0              3  [Timestamp('2010-05-30 06:17:57+0000', tz='UTC')]   
4        0              4  [Timestamp('2010-05-31 20:10:37+0000', tz='UTC...   

                                           locations            day_of_week  \
0  [608105, 8977, 18574, 17269, 1161876, 1163401,...  [5, 5, 5, 6, 6, 6, 6]   
1                                            [21714]                    [2]   
2                                           [420315]                    [3]   
3                                             [9073]                    [6]   
4                     [18417, 480992, 15326, 420315]           [0, 1, 2, 2]   

                   hour_of_day              

In [7]:
# Random subset of users
unique_users = trajectories_df['user_id'].unique()
random.seed(42)
selected_users = random.sample(list(unique_users), 201)

filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].reset_index(drop=True)

# Label encode user_id
label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])
num_users = len(label_encoder.classes_)
print(f"Number of selected users: {num_users}")

train_df, val_df = train_test_split(filtered_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


Number of selected users: 201


In [8]:
def build_mixed_trajectory(loc_ids, days, hours, lat_buckets, lon_buckets):
    """
    Convert each check-in to a single token that includes 
    (POI, day_of_week, hour_of_day, lat_bucket, lon_bucket).
    e.g., "POI123_DAY2_H14_LAT39_LON116"
    """
    tokens = []
    for loc_id, d, h, latb, lonb in zip(loc_ids, days, hours, lat_buckets, lon_buckets):
        token = f"POI{loc_id}_DAY{d}_H{h}_LAT{latb}_LON{lonb}"
        tokens.append(token)
    return " ".join(tokens)


In [9]:
class SpatioTemporalGowallaDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=128):
        """
        df should contain:
          - encoded_locations (list of location IDs)
          - day_of_week (list of day indices)
          - hour_of_day (list of hour indices)
          - lat_bucket (list of lat bucket ints)
          - lon_bucket (list of lon bucket ints)
          - label (integer-encoded user ID)
        """
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.labels = df["label"].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        loc_ids = row["encoded_locations"]
        days = row["day_of_week"]
        hours = row["hour_of_day"]
        latb = row["lat_bucket"]
        lonb = row["lon_bucket"]

        # Build spatio-temporal token string
        trajectory_str = build_mixed_trajectory(loc_ids, days, hours, latb, lonb)

        # Tokenize
        inputs = self.tokenizer(
            trajectory_str,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt",
        )

        label = self.labels[idx]

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'label': torch.tensor(label, dtype=torch.long)
        }


In [10]:
# Instantiate the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create dataset objects
train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=128)
val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=128)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

class TrajectoryDistilBERT(nn.Module):
    def __init__(self, num_users, dropout_rate=0.1):
        super(TrajectoryDistilBERT, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.dim, num_users)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits




tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# Initialize model
model = TrajectoryDistilBERT(num_users)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer and loss
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = nn.CrossEntropyLoss()


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
def compute_metrics(outputs, labels, top_ks=[1, 5]):
    metrics = {}
    with torch.no_grad():
        for k in top_ks:
            _, topk_indices = torch.topk(outputs, k=k, dim=1)
            correct_topk = 0
            for i in range(labels.size(0)):
                if labels[i].item() in topk_indices[i]:
                    correct_topk += 1
            metrics[f"ACC@{k}"] = correct_topk / labels.size(0)

    # For F1 and classification report
    _, preds = torch.max(outputs, dim=1)
    metrics["top1_preds"] = preds
    return metrics

from sklearn.metrics import classification_report

def evaluate_model(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []
    acc_metrics = {"ACC@1": 0, "ACC@5": 0}
    total_samples = 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size

            batch_metrics = compute_metrics(outputs, labels)
            acc_metrics["ACC@1"] += batch_metrics["ACC@1"] * batch_size
            acc_metrics["ACC@5"] += batch_metrics["ACC@5"] * batch_size

            all_preds.extend(batch_metrics["top1_preds"].cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_samples += batch_size

    avg_loss = total_loss / total_samples
    acc_metrics["ACC@1"] /= total_samples
    acc_metrics["ACC@5"] /= total_samples

    classification_metrics = classification_report(all_labels, all_preds, output_dict=True)
    return avg_loss, acc_metrics, classification_metrics


In [14]:
def train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=5):
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        train_iter = tqdm(train_loader, desc=f"Epoch {epoch+1} [Training]", leave=False)
        
        for batch in train_iter:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            train_iter.set_description(f"Epoch {epoch+1} [Training] loss: {loss.item():.4f}")

        # Evaluate on train and val sets
        train_loss, train_acc_metrics, train_classification_metrics = evaluate_model(model, train_loader, device, loss_fn)
        val_loss, val_acc_metrics, val_classification_metrics = evaluate_model(model, val_loader, device, loss_fn)

        # Print epoch summary
        print(f"\nEpoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Train ACC@1: {train_acc_metrics['ACC@1']:.4f}, ACC@5: {train_acc_metrics['ACC@5']:.4f}")
        print(f"  Val   Loss: {val_loss:.4f},   Val ACC@1:   {val_acc_metrics['ACC@1']:.4f}, ACC@5: {val_acc_metrics['ACC@5']:.4f}")
        print(f"  Val Macro F1: {val_classification_metrics['macro avg']['f1-score']:.4f}, Weighted F1: {val_classification_metrics['weighted avg']['f1-score']:.4f}")


In [14]:
# epochs = 1
# train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)


In [21]:
filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].copy()
filtered_df.reset_index(drop=True, inplace=True)

label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])


# Grid search for finetuning

In [16]:
def run_experiment(
    dropout_rate, 
    lr, 
    batch_size, 
    max_length,
    train_df,
    val_df,
    epochs=5,
    device='cuda'
):
    # 1. Recreate datasets/loaders for each batch_size/max_length
    train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=max_length)
    val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=max_length)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # 2. Initialize the model with dropout
    num_users = len(label_encoder.classes_)
    model = TrajectoryDistilBERT(num_users, dropout_rate=dropout_rate)
    model.to(device)

    # 3. Create optimizer and loss
    optimizer = AdamW(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    # 4. Train
    train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)

    # 5. Evaluate final metrics
    final_loss, acc_metrics, cls_report = evaluate_model(model, val_loader, device, loss_fn)
    
    # 6. Return  metrics
    return {
        "final_loss": final_loss,
        "ACC@1": acc_metrics["ACC@1"],
        "ACC@5": acc_metrics["ACC@5"],
        "macro_f1": cls_report["macro avg"]["f1-score"]
    }


In [17]:
selected_users_df = gowalla_data[gowalla_data["user_id"].isin(selected_users)].copy()

# Label-encode user_id again in this filtered dataframe
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
selected_users_df["label"] = label_encoder.fit_transform(selected_users_df["user_id"])


In [18]:
import itertools

param_grid = {
    "dropout_rate": [0.0, 0.1],
    "lr": [2e-5, 3e-5],
    "batch_size": [16, 32],
    "max_length": [64, 128]
}

keys = list(param_grid.keys())
values = list(param_grid.values())

results = []
for combo in itertools.product(*values):  # All combos
    params = dict(zip(keys, combo))  
    print(f"\nRunning experiment with {params}")

    metrics = run_experiment(
        dropout_rate=params["dropout_rate"],
        lr=params["lr"],
        batch_size=params["batch_size"],
        max_length=params["max_length"],
        train_df=train_df,  
        val_df=val_df,
        epochs=5,  # or 10
        device=device
    )

    # Store the results
    all_results = {**params, **metrics}
    results.append(all_results)

    print(f"Metrics => {metrics}")



Running experiment with {'dropout_rate': 0.0, 'lr': 2e-05, 'batch_size': 16, 'max_length': 64}


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifica


Epoch 1:
  Train Loss: 4.1250, Train ACC@1: 0.1978, ACC@5: 0.4458
  Val   Loss: 4.2373,   Val ACC@1:   0.1603, ACC@5: 0.4129
  Val Macro F1: 0.0356, Weighted F1: 0.0884


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 2:
  Train Loss: 2.4463, Train ACC@1: 0.5603, ACC@5: 0.8157
  Val   Loss: 2.6533,   Val ACC@1:   0.5261, ACC@5: 0.7666
  Val Macro F1: 0.2645, Weighted F1: 0.4264


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 4:
  Train Loss: 1.4715, Train ACC@1: 0.6893, ACC@5: 0.9237
  Val   Loss: 1.7448,   Val ACC@1:   0.6359, ACC@5: 0.8728
  Val Macro F1: 0.4268, Weighted F1: 0.5722


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 5:
  Train Loss: 1.2856, Train ACC@1: 0.7120, ACC@5: 0.9359
  Val   Loss: 1.6038,   Val ACC@1:   0.6376, ACC@5: 0.8763
  Val Macro F1: 0.4429, Weighted F1: 0.5875
Metrics => {'final_loss': 1.603756724749708, 'ACC@1': 0.6376306620209059, 'ACC@5': 0.8763066202090593, 'macro_f1': 0.44286839678279816}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [19]:
best = max(results, key=lambda x: x["ACC@1"])  # or "macro_f1", etc.
print("Best config by ACC@1:", best)


Best config by ACC@1: {'dropout_rate': 0.0, 'lr': 3e-05, 'batch_size': 16, 'max_length': 128, 'final_loss': 1.3637361680173707, 'ACC@1': 0.6951219512195121, 'ACC@5': 0.9059233449477352, 'macro_f1': 0.5092049996654855}


# Using best parameters for 201 users


In [55]:
# Random subset of users
unique_users = trajectories_df['user_id'].unique()
random.seed(42)
selected_users = random.sample(list(unique_users), 201)

filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].reset_index(drop=True)

# Label encode user_id
label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])
num_users = len(label_encoder.classes_)
print(f"Number of selected users: {num_users}")

train_df, val_df = train_test_split(filtered_df, test_size=0.2, random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)


Number of selected users: 201


In [56]:
# Best Hyperparams
best_dropout = 0.0
best_lr = 3e-5
best_batch_size = 16
best_max_length = 128

train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=best_max_length)
val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=best_max_length)

train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)


In [57]:
class TrajectoryDistilBERT(nn.Module):
    def __init__(self, num_users, dropout_rate=0.1):
        super(TrajectoryDistilBERT, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.dim, num_users)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # DistilBERT's last_hidden_state: (batch_size, seq_len, hidden_size)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits


In [58]:
# Build the model 
model = TrajectoryDistilBERT(num_users, dropout_rate=best_dropout)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

TrajectoryDistilBERT(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [59]:
optimizer = AdamW(model.parameters(), lr=best_lr)
loss_fn = nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [60]:
epochs = 10  
train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 1:
  Train Loss: 3.1399, Train ACC@1: 0.3669, ACC@5: 0.6675
  Val   Loss: 3.2669,   Val ACC@1:   0.3519, ACC@5: 0.6394
  Val Macro F1: 0.1179, Weighted F1: 0.2472


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 2:
  Train Loss: 1.9786, Train ACC@1: 0.6113, ACC@5: 0.8588
  Val   Loss: 2.2305,   Val ACC@1:   0.5505, ACC@5: 0.8171
  Val Macro F1: 0.3110, Weighted F1: 0.4667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 9:
  Train Loss: 0.5543, Train ACC@1: 0.8667, ACC@5: 0.9847
  Val   Loss: 1.2026,   Val ACC@1:   0.7160, ACC@5: 0.9059
  Val Macro F1: 0.5453, Weighted F1: 0.6839


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 10:
  Train Loss: 0.4668, Train ACC@1: 0.8924, ACC@5: 0.9887
  Val   Loss: 1.2117,   Val ACC@1:   0.7160, ACC@5: 0.9077
  Val Macro F1: 0.5543, Weighted F1: 0.6908


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

# WITH TEST SET

In [17]:
# Random subset of users
unique_users = trajectories_df['user_id'].unique()
random.seed(42)
selected_users = random.sample(list(unique_users), 291)

filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].reset_index(drop=True)

# Label encode user_id
label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])
num_users = len(label_encoder.classes_)
print(f"Number of selected users: {num_users}")

# Split data into training, validation, and test sets
train_df, temp_df = train_test_split(filtered_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset indices for all sets
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Best Hyperparams
best_dropout = 0.0
best_lr = 3e-5
best_batch_size = 16
best_max_length = 128

# Create datasets
train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=best_max_length)
val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=best_max_length)
test_dataset = SpatioTemporalGowallaDataset(test_df, tokenizer, max_length=best_max_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

def evaluate_test_set(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []
    acc_metrics = {"ACC@1": 0, "ACC@5": 0}
    total_samples = 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size

            batch_metrics = compute_metrics(outputs, labels)
            acc_metrics["ACC@1"] += batch_metrics["ACC@1"] * batch_size
            acc_metrics["ACC@5"] += batch_metrics["ACC@5"] * batch_size

            all_preds.extend(batch_metrics["top1_preds"].cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_samples += batch_size

    # Compute averages
    avg_loss = total_loss / total_samples
    acc_metrics["ACC@1"] /= total_samples
    acc_metrics["ACC@5"] /= total_samples

    # Generate classification report
    cls_metrics = classification_report(all_labels, all_preds, output_dict=True)
    
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"ACC@1: {acc_metrics['ACC@1']:.4f}")
    print(f"ACC@5: {acc_metrics['ACC@5']:.4f}")
    
    return avg_loss, acc_metrics, cls_metrics



# Build the model
model = TrajectoryDistilBERT(num_users, dropout_rate=best_dropout)
model.to(device)

# Optimizer, loss function, and training
optimizer = AdamW(model.parameters(), lr=best_lr)
loss_fn = nn.CrossEntropyLoss()

# Train the model
epochs = 10
train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)

# Evaluate the test set
print("Evaluating the model on the test set...")
evaluate_test_set(model, test_loader, device, loss_fn)


Number of selected users: 291


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifica


Epoch 1:
  Train Loss: 4.4952, Train ACC@1: 0.1168, ACC@5: 0.3108
  Val   Loss: 4.5678,   Val ACC@1:   0.1179, ACC@5: 0.3134
  Val Macro F1: 0.0197, Weighted F1: 0.0469


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 2:
  Train Loss: 3.5636, Train ACC@1: 0.2367, ACC@5: 0.5471
  Val   Loss: 3.6956,   Val ACC@1:   0.2213, ACC@5: 0.5331
  Val Macro F1: 0.0534, Weighted F1: 0.1158


...


Epoch 9:
  Train Loss: 1.6590, Train ACC@1: 0.5461, ACC@5: 0.8732
  Val   Loss: 2.0984,   Val ACC@1:   0.4847, ACC@5: 0.8094
  Val Macro F1: 0.3109, Weighted F1: 0.4121


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 10:
  Train Loss: 1.5737, Train ACC@1: 0.5662, ACC@5: 0.8846
  Val   Loss: 2.0550,   Val ACC@1:   0.5024, ACC@5: 0.8126
  Val Macro F1: 0.3187, Weighted F1: 0.4280
Evaluating the model on the test set...
Test Loss: 2.0699
ACC@1: 0.4976
ACC@5: 0.8174


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

(2.0699333462846106,
 {'ACC@1': 0.4975767366720517, 'ACC@5': 0.8174474959612278},
 {'1': {'precision': 0.45,
   'recall': 0.6923076923076923,
   'f1-score': 0.5454545454545455,
   'support': 13},
  '2': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2},
  '3': {'precision': 0.5,
   'recall': 1.0,
   'f1-score': 0.6666666666666666,
   'support': 3},
  '4': {'precision': 0.25,
   'recall': 0.2,
   'f1-score': 0.22222222222222224,
   'support': 5},
  '5': {'precision': 0.5555555555555556,
   'recall': 0.4166666666666667,
   'f1-score': 0.4761904761904762,
   'support': 12},
  '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 2},
  '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 12},
  '9': {'precision': 0.38095238095238093,
   'recall': 0.5714285714285714,
   'f1-score': 0.4571428571428571,
   'support': 14},
  '10': {'precision': 1.0,
   'recall': 0.5,
   'f1-score': 0.6666666666666666,
   'support': 2},
  '11': {'precision': 0.0, 'recal

# Using best parameters for 112 users

In [15]:
# Random subset of users
unique_users = trajectories_df['user_id'].unique()
random.seed(42)
selected_users = random.sample(list(unique_users), 112)

filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].reset_index(drop=True)

# Label encode user_id
label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])
num_users = len(label_encoder.classes_)
print(f"Number of selected users: {num_users}")

# Split data into training, validation, and test sets
train_df, temp_df = train_test_split(filtered_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset indices for all sets
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)



Number of selected users: 112


In [16]:
# Best Hyperparams
best_dropout = 0.0
best_lr = 3e-5
best_batch_size = 16
best_max_length = 128

train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=best_max_length)
val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=best_max_length)
test_dataset = SpatioTemporalGowallaDataset(test_df, tokenizer, max_length=best_max_length)


train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)



In [ ]:
def evaluate_test_set(model, test_loader, device):
    model.eval()
    total_loss, total_correct = 0, 0
    all_predictions, all_labels = [], []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = loss_fn(outputs, labels)

            total_loss += loss.item()
            predictions = torch.argmax(outputs, dim=1)
            total_correct += (predictions == labels).sum().item()

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = total_correct / len(test_loader.dataset)
    print(f"Test Loss: {total_loss / len(test_loader):.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(classification_report(all_labels, all_predictions))

# Evaluate the test set
evaluate_test_set(model, test_loader, device)


In [51]:
class TrajectoryDistilBERT(nn.Module):
    def __init__(self, num_users, dropout_rate=0.1):
        super(TrajectoryDistilBERT, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.bert.config.dim, num_users)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        # DistilBERT's last_hidden_state: (batch_size, seq_len, hidden_size)
        cls_output = outputs.last_hidden_state[:, 0, :]
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits


In [52]:
# Build the model
model = TrajectoryDistilBERT(num_users, dropout_rate=best_dropout)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

TrajectoryDistilBERT(
  (bert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Li

In [53]:
optimizer = AdamW(model.parameters(), lr=best_lr)
loss_fn = nn.CrossEntropyLoss()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [54]:
epochs = 10  
train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 1:
  Train Loss: 3.4679, Train ACC@1: 0.3078, ACC@5: 0.5286
  Val   Loss: 3.5049,   Val ACC@1:   0.3368, ACC@5: 0.5193
  Val Macro F1: 0.0773, Weighted F1: 0.2083


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 2:
  Train Loss: 1.8978, Train ACC@1: 0.6183, ACC@5: 0.8742
  Val   Loss: 2.0399,   Val ACC@1:   0.6070, ACC@5: 0.8246
  Val Macro F1: 0.3052, Weighted F1: 0.5294


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 9:
  Train Loss: 0.5440, Train ACC@1: 0.8531, ACC@5: 0.9894
  Val   Loss: 1.0333,   Val ACC@1:   0.7228, ACC@5: 0.9368
  Val Macro F1: 0.5455, Weighted F1: 0.6802


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Epoch 10:
  Train Loss: 0.4714, Train ACC@1: 0.8760, ACC@5: 0.9921
  Val   Loss: 1.0010,   Val ACC@1:   0.7404, ACC@5: 0.9509
  Val Macro F1: 0.5813, Weighted F1: 0.6931


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

# WITH TEST SET

In [18]:
# Random subset of users
unique_users = trajectories_df['user_id'].unique()
random.seed(42)
selected_users = random.sample(list(unique_users), 112)

filtered_df = trajectories_df[trajectories_df['user_id'].isin(selected_users)].reset_index(drop=True)

# Label encode user_id
label_encoder = LabelEncoder()
filtered_df["label"] = label_encoder.fit_transform(filtered_df["user_id"])
num_users = len(label_encoder.classes_)
print(f"Number of selected users: {num_users}")

# Split data into training, validation, and test sets
train_df, temp_df = train_test_split(filtered_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Reset indices for all sets
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# Best Hyperparams
best_dropout = 0.0
best_lr = 3e-5
best_batch_size = 16
best_max_length = 128

# Create datasets
train_dataset = SpatioTemporalGowallaDataset(train_df, tokenizer, max_length=best_max_length)
val_dataset = SpatioTemporalGowallaDataset(val_df, tokenizer, max_length=best_max_length)
test_dataset = SpatioTemporalGowallaDataset(test_df, tokenizer, max_length=best_max_length)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=best_batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=best_batch_size, shuffle=False)

def evaluate_test_set(model, loader, device, loss_fn):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []
    acc_metrics = {"ACC@1": 0, "ACC@5": 0}
    total_samples = 0

    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            batch_size = labels.size(0)
            total_loss += loss.item() * batch_size

            batch_metrics = compute_metrics(outputs, labels)
            acc_metrics["ACC@1"] += batch_metrics["ACC@1"] * batch_size
            acc_metrics["ACC@5"] += batch_metrics["ACC@5"] * batch_size

            all_preds.extend(batch_metrics["top1_preds"].cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            total_samples += batch_size

    # Compute averages
    avg_loss = total_loss / total_samples
    acc_metrics["ACC@1"] /= total_samples
    acc_metrics["ACC@5"] /= total_samples

    # Generate classification report
    cls_metrics = classification_report(all_labels, all_preds, output_dict=True)
    
    print(f"Test Loss: {avg_loss:.4f}")
    print(f"ACC@1: {acc_metrics['ACC@1']:.4f}")
    print(f"ACC@5: {acc_metrics['ACC@5']:.4f}")
    
    return avg_loss, acc_metrics, cls_metrics

# Build the model
model = TrajectoryDistilBERT(num_users, dropout_rate=best_dropout)
model.to(device)

# Optimizer, loss function, and training
optimizer = AdamW(model.parameters(), lr=best_lr)
loss_fn = nn.CrossEntropyLoss()

# Train the model
epochs = 10  
train_model(model, train_loader, val_loader, device, loss_fn, optimizer, epochs=epochs)

# Evaluate the test set
print("Evaluating the model on the test set...")
evaluate_test_set(model, test_loader, device, loss_fn)


Number of selected users: 112


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classifica


Epoch 1:
  Train Loss: 3.9666, Train ACC@1: 0.1286, ACC@5: 0.3920
  Val   Loss: 4.0299,   Val ACC@1:   0.1080, ACC@5: 0.3662
  Val Macro F1: 0.0089, Weighted F1: 0.0234


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 2:
  Train Loss: 3.0880, Train ACC@1: 0.2704, ACC@5: 0.6472
  Val   Loss: 3.1399,   Val ACC@1:   0.2488, ACC@5: 0.6479
  Val Macro F1: 0.0697, Weighted F1: 0.1212


...


Epoch 9:
  Train Loss: 1.1317, Train ACC@1: 0.6814, ACC@5: 0.9487
  Val   Loss: 1.4359,   Val ACC@1:   0.6056, ACC@5: 0.8967
  Val Macro F1: 0.3767, Weighted F1: 0.5150


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif


Epoch 10:
  Train Loss: 1.0609, Train ACC@1: 0.6884, ACC@5: 0.9608
  Val   Loss: 1.3931,   Val ACC@1:   0.5869, ACC@5: 0.9014
  Val Macro F1: 0.3509, Weighted F1: 0.5072
Evaluating the model on the test set...
Test Loss: 1.6244
ACC@1: 0.6262
ACC@5: 0.8411


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

(1.6243614481988353,
 {'ACC@1': 0.6261682242990654, 'ACC@5': 0.8411214953271028},
 {'1': {'precision': 0.8,
   'recall': 1.0,
   'f1-score': 0.888888888888889,
   'support': 8},
  '3': {'precision': 0.6666666666666666,
   'recall': 0.8571428571428571,
   'f1-score': 0.75,
   'support': 7},
  '4': {'precision': 0.125,
   'recall': 0.3333333333333333,
   'f1-score': 0.18181818181818182,
   'support': 3},
  '7': {'precision': 0.6666666666666666,
   'recall': 1.0,
   'f1-score': 0.8,
   'support': 4},
  '8': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 1},
  '10': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1},
  '11': {'precision': 0.6666666666666666,
   'recall': 1.0,
   'f1-score': 0.8,
   'support': 2},
  '12': {'precision': 1.0,
   'recall': 0.75,
   'f1-score': 0.8571428571428571,
   'support': 8},
  '13': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 1},
  '14': {'precision': 0.4,
   'recall': 1.0,
   'f1-score': 0.5714285714285715,